In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_excel('99acres_Bangalore_scraped_data/99acres_scraped_data.xlsx')

In [3]:
df.head()

,Region,Link,Bedroom,Bathroom,Balcony,Additional_rooms,Area,Facing,Furnishing,Rating,...,Posted_By_and_On,Age,Pet_friendly,Floor,Available_from,Available_for,Type,Advance,Location,Rent
0,Bangalore Central,https://www.99acres.com/2-bhk-bedroom-apartmen...,2 Bedrooms,2 Bathrooms,1 Balcony,NaN,1400.0,East,Semifurnished,4.3,...,"Dealer on Feb 22, 2025",0 to 1 Year Old,NaN,3rd of 5 Floors,Immediate,All,Flat/Apartment for Rent,"Rate per sq.ft. ₹ 0 per sq.ft. Brokerage₹ 54,0...","in Wellington Park, Richmond Town, Bangalore","54,000"
1,Bangalore Central,https://www.99acres.com/2-bhk-bedroom-independ...,2 Bedrooms,2 Bathrooms,1 Balcony,Pooja Room,1300.0,East,Semifurnished,4.3,...,"Owner on Feb 21, 2025",10+ Year Old,NaN,2nd of 3 Floors,Immediate,Family Only,Independent/Builder Floor for Rent,Rate per sq.ft. ₹ 0 per sq.ft. Deposit10 month...,"in Wilson Garden, Bangalore","28,000"
2,Bangalore Central,https://www.99acres.com/2-bhk-bedroom-independ...,2 Bedrooms,2 Bathrooms,1 Balcony,Others,900.0,East,Semifurnished,4.2,...,"Dealer on Feb 20, 2025",1 to 5 Year Old,NaN,3rd of 4 Floors,Immediate,Family Only,Independent/Builder Floor for Rent,NaN,"in Victoria Layout, Ashok Nagar",25 Lac
3,Bangalore Central,https://www.99acres.com/1-bhk-bedroom-independ...,1 Bedroom,1 Bathroom,1 Balcony,NaN,700.0,North-East,Furnished,4.2,...,"Dealer on Feb 19, 2025",5 to 10 Year Old,NaN,2 Floors,Immediate,"Family , Bachelors (Men Only)",Independent House/Villa for Rent,"Rate per sq.ft. ₹ 0 per sq.ft. Brokerage₹ 18,0...","in Vasanth nagar, Bangalore","18,000"
4,Bangalore Central,https://www.99acres.com/3-bhk-bedroom-independ...,3 Bedrooms,3 Bathrooms,2 Balconies,NaN,1509.0,North-East,Semifurnished,4.2,...,"Dealer on Feb 19, 2025",5 to 10 Year Old,NaN,2 Floors,Immediate,"Family , Bachelors (Men Only)",Independent House/Villa for Rent,"Rate per sq.ft. ₹ 0 per sq.ft. Brokerage₹ 40,0...","in Vasanth nagar, Bangalore","40,000"


In [4]:
df_copy = df.copy()
df_copy = df_copy.dropna(subset=['Rent' , 'Bathroom' , 'Bedroom' , 'Available_for' , 'Furnishing'] , how = 'any').reset_index(drop= True)

### Bedroom

In [5]:
df_copy['Bedroom'] = df_copy['Bedroom'].str.findall(r'^\d+').str.get(0).fillna(0).astype(float)

In [6]:
df_copy['Bedroom'].value_counts()

Bedroom
2.0     351
3.0     281
1.0     163
4.0      72
5.0       4
10.0      1
Name: count, dtype: int64

In [7]:
df_copy = df_copy.iloc[np.array(df_copy['Bedroom'] <= 6) , :].reset_index(drop = True)

### Bathroom

In [8]:
df_copy['Bathroom'] = df_copy['Bathroom'].str.findall(r'^\d+').str.get(0).fillna(0).astype(float)

In [9]:
df_copy['Bathroom'].value_counts()

Bathroom
2.0    383
3.0    196
1.0    190
4.0     78
5.0     22
7.0      1
6.0      1
Name: count, dtype: int64

In [10]:
df_copy = df_copy.iloc[np.array(df_copy['Bathroom'] <= 6) , :].reset_index(drop = True)

### Balcony

In [11]:
df_copy['Balcony'] = df_copy['Balcony'].str.findall(r'^\d+\+?').str.get(0).fillna('0')

In [12]:
df_copy['Balcony'].value_counts()

Balcony
1     340
2     253
0     172
3      78
3+     27
Name: count, dtype: int64

### Additional Room

In [13]:
df_copy['Additional_rooms'] = df_copy['Additional_rooms'].apply(lambda x: len(x.split(',')) if pd.notna(x) else None).fillna(0)

In [14]:
df_copy['Additional_rooms'].value_counts()

Additional_rooms
0.0    499
1.0    243
2.0     66
4.0     31
3.0     31
Name: count, dtype: int64

In [15]:
df_copy = df_copy.iloc[np.array(df_copy['Additional_rooms'] != 5) , :].reset_index(drop = True)

### Rent

In [16]:
df_copy = df_copy.iloc[~np.array(df_copy['Rent'] == 'Price on Request') , :].reset_index(drop = True)

In [17]:
def Lac(x):
    if 'Lac' in x:
        return np.round(float(re.search(r'[\d\.]+' , x).group()) * 100000 , 1)
    else:
            return np.round(float(x) , 1)

In [18]:
df_copy['Rent'] = df_copy['Rent'].str.split(',').apply(lambda x : ''.join(x)).apply(lambda x : Lac(x))

In [19]:
df_copy['Rent'].describe().round(2)

count        868.00
mean       77957.46
std       247882.16
min         4000.00
25%        21000.00
50%        35600.00
75%        60500.00
max      3500000.00
Name: Rent, dtype: float64

In [20]:
df_copy = df_copy.loc[df_copy['Rent'] <= 500000 , :].reset_index(drop = True)

### Area (sq.ft)

In [21]:
df_copy.rename(columns= {'Area' : 'Area (sq.ft)'} , inplace=True)

In [22]:
df_copy = df_copy.loc[df_copy['Area (sq.ft)'] <= 10000 , :].reset_index(drop = True)

In [23]:
df_copy['Area (sq.ft)'].describe().round(2)

count      856.00
mean      1460.50
std        975.16
min         30.00
25%        900.00
50%       1200.00
75%       1767.89
max      10000.00
Name: Area (sq.ft), dtype: float64

### Brokerage

In [24]:
df_copy['Brokerage'] = df_copy['Advance'].str.findall(r'Brokerage₹\s[\d\,]+|Brokerage\d+\%').str.get(0)

In [25]:
def Brokerage(x):
    try:
        if '%' in x['Brokerage']:
            return np.round((float(re.search(r'\d+' , x['Brokerage']).group()) * x['Rent'])/100 , 1)
        else:
            return np.round(float(''.join(re.search(r'[\d\,]+' , x['Brokerage']).group().split(','))) , 1)
    except:
        return 0

In [26]:
df_copy['Brokerage'] = df_copy.apply(Brokerage , axis=1)

In [27]:
df_copy['Brokerage'].describe().round(2)

count        856.00
mean       34637.35
std        65043.30
min            0.00
25%            0.00
50%           30.00
75%        50000.00
max      1050000.00
Name: Brokerage, dtype: float64

In [28]:
df_copy = df_copy.loc[df_copy['Brokerage'] <= 1400000 , :].reset_index(drop = True)

### Deposit

In [29]:
df_copy['Deposit'] = df_copy['Advance'].str.findall(r'Deposit₹\s[\d\,]+|Deposit\d+\smonths').str.get(0)

In [30]:
def Deposit(x):
    try:
        if 'months' in x['Deposit']:
            return np.round((float(re.search(r'\d+' , x['Deposit']).group()) * x['Rent']) , 1)
        else:
            return np.round(float(''.join(re.search(r'[\d\,]+' , x['Deposit']).group().split(','))) , 1)
    except:
        return 0

In [31]:
df_copy['Deposit'] = df_copy.apply(Deposit , axis=1)

In [32]:
df_copy['Deposit'].describe().round(2)

count        856.00
mean      284972.49
std       433779.74
min            0.00
25%        40000.00
50%       150000.00
75%       300000.00
max      4000000.00
Name: Deposit, dtype: float64

In [33]:
df_copy = df_copy.loc[df_copy['Deposit'] <= 5000000 , :].reset_index(drop = True)

### Maintenance

In [34]:
def Maintenance(x):
    try: 
        return np.round(float(''.join(x)) , 1)
    except:
        return 0

In [35]:
df_copy['Maintenance'] = df_copy['Advance'].str.findall(r'Maintenance₹\s[\d\,]+').str.get(0).str.findall(r'[\d\,]+').str.get(0).str.split(',').apply(Maintenance)

In [36]:
df_copy = df_copy.iloc[np.array(df_copy['Maintenance'] < df_copy['Rent']) , :].reset_index()

In [37]:
df_copy = df_copy.loc[df_copy['Maintenance'] <= 75000 , :].reset_index(drop = True)

### Covered Parking

In [38]:
df_copy['Covered_Parking'] = df_copy['Parking'].str.findall(r'\d+\sCovered').str.get(0).str.findall(r'\d+').str.get(0).fillna(0).astype('float')

In [39]:
df_copy = df_copy.loc[df_copy['Covered_Parking'] <= 6 , :].reset_index(drop = True)

### Age

In [40]:
def Age(x):
    try:
        if 'to' in x:
            return np.round(float(re.search(r'\d+' , x).group()))
        elif 'Under Construction' == x:
            return 0
        else:
            return x
    except:
        return None

In [41]:
df_copy['Age'] = df_copy['Age'].str.findall(r'to\s\d+|\d+\+|Under\sConstruction').str.get(0).apply(Age)

In [42]:
df_copy = df_copy.dropna(subset=['Age']).reset_index(drop=True)

In [43]:
df_copy.loc[df_copy['Age'] == '10+' , 'Age'] = 11
df_copy['Age'] = df_copy['Age'].astype(float)

In [44]:
def categorize_age(value):
    if pd.isna(value):
        return "Undefined"
    if value == 0:
        return "Under Construction"
    if 1 <= value <= 5:
        return "New Property"
    if 6 <= value <= 10:
        return "Moderately Old"
    if value > 10:
        return "Old Property"
    else:
        return "Undefined"

df_copy['Age'] = df_copy['Age'].apply(categorize_age)

In [45]:
df_copy['Age'].value_counts()

Age
New Property          578
Moderately Old        182
Old Property           74
Under Construction      8
Name: count, dtype: int64

### Type

In [46]:
df_copy['Type'] = df_copy['Type'].replace({
    'Flat/Apartment for Rent': 'Apartment',
    'Independent House/Villa for Rent': 'House/Villa',
    'Independent/Builder Floor for Rent': 'Builder Floor',
    'Studio Apartment for Rent': 'Studio',
    'Serviced Apartments for Rent': 'Serviced Apartment'
})

### Total Floors

In [47]:
df_copy['Total_Floors'] = df_copy['Floor'].str.findall(r'\d+\sFloors').str.get(0).str.findall(r'\d+').str.get(0).fillna(0).astype(float)

In [48]:
df_copy = df_copy[~(df_copy['Total_Floors'] == 0)].reset_index(drop=True)

In [49]:
def categorize_floors(floor):
    if floor <= 3:
        return 'Low-rise'
    elif 4 <= floor <= 10:
        return 'Mid-rise'
    elif 11 <= floor <= 20:
        return 'High-rise'
    else:
        return 'Skyscraper'

df_copy['Total_Floors'] = df_copy['Total_Floors'].apply(categorize_floors)

In [50]:
df_copy['Total_Floors'].value_counts()

Total_Floors
Mid-rise      322
Low-rise      253
High-rise     144
Skyscraper    110
Name: count, dtype: int64

In [51]:
df_copy = df_copy[['Region', 'Bedroom', 'Bathroom', 'Balcony', 'Additional_rooms', 'Area (sq.ft)', 'Furnishing', 'Age' ,'Covered_Parking', 'Brokerage', 'Deposit', 'Maintenance', 'Type', 'Total_Floors', 'Rent']]

In [52]:
df_copy = df_copy.drop_duplicates().reset_index(drop = True)

In [53]:
df_copy.shape

(768, 15)

In [54]:
df_copy.to_excel("Cleaned_Bangalore_House_Rent_Test_Data.xlsx" , index=False)

#### Model Prediction

In [55]:
import pickle
with open('House_Rent_Prediction_Pipeline.pkl','rb') as file:
    pipeline = pickle.load(file)

In [56]:
x = df_copy.drop(columns=['Rent'])
y = df_copy['Rent']

In [57]:
y_pred = np.expm1(pipeline.predict(x))

In [58]:
total = 0
for i in range(y.shape[0]):
    if y_pred[i] - 6000 <= y[i] <= y_pred[i] + 6000:
        total += 1

total / y.shape[0]

0.7200520833333334

#### R2 Score

In [59]:
from sklearn.metrics import r2_score

r2_score(y , y_pred)

0.8703656374921901